In [1]:
import os

from salespy import SalesforceClass, EpicorClass

# login to Salesforce & get credentials
sf = SalesforceClass(
    username='bko@fs-elliott.com.test', sandbox='test',
    org_id='00D7A0000009Kmw'
)
creds = sf.login()
print(f'--- Logged Into Salesforce ---')

# get most recent Tran Num for filtering Epicor report
last_tran_num = sf.get_last_tran(creds=creds)
print(f'--- Got Last Tran Num: {last_tran_num} ---')

# get Epicor file
# ----- TODO: Loop through files and delete ----- #
directory = (
    '/Users/beekayokay/OneDrive/Projects/'
    'Salesforce Epicor Integration/'
)
file_name = 'test_bookings_df_2.xlsx'
epicor = EpicorClass()
bookings_df = epicor.bookings_to_df(os.path.join(directory, file_name))
print(f'--- Got Epicor Data: {bookings_df.size} ---')

# remove existing SFDC Tran Nums
old_trans = bookings_df['Name'] <= last_tran_num
bookings_df = bookings_df[~old_trans]
print(f'--- Removed Existing Epicor Data: {bookings_df.size} ---')

# get tuples of Order Num Ln
order_num_ln = tuple(bookings_df['Order_Num_Ln__c'])
print(f'--- Compiled Order Num Ln ---')

# get Salesforce records to append
sfdc_df = sf.get_epicor_trans(creds=creds, order_num_ln=order_num_ln)
print(f'--- Got Matching SFDC Data: {sfdc_df.size} ---')

# add fo and asset
bookings_df = sf.add_sn_fo(creds=creds, df=bookings_df)
bookings_df = sf.add_sn_asset(creds=creds, df=bookings_df)
print(f'--- Added FO & SN: {bookings_df.size} ---')

--- Logged Into Salesforce ---
--- Got Last Tran Num: 4665.0 ---
--- Got Epicor Data: 660 ---
--- Removed Existing Epicor Data: 616 ---
--- Compiled Order Num Ln ---
--- Got Matching SFDC Data: 230 ---
--- Added FO & SN: 644 ---


In [3]:
bookings_df

,OrderNum__c,OrderLn__c,Name,Book_Date__c,BookType__c,OrderRel__c,Prod_Code__c,SN__c,Order_Line_Amt__c,PO_Num__c,...,Ship_To_Zip__c,Ship_To_Country__c,Packager__c,Market_Segment__c,Payment_Terms__c,OrderHed_FOB__c,SysRowID__c,Order_Num_Ln__c,Factory_Order__c,Asset__c
0,1000,1,4682,2003-12-30,Void,1,AIIE,E110042-3,0.00,A009-41003 R1,...,NaN,INDONESIA,NaN,NaN,XLETTER OF CRED,C&F JAKARTA,92059e56-e620-40ac-e111-e6c2e2812162,1000-1,NaN,02ii0000006PvLyAAK
1,1001,1,4683,2003-10-02,New,1,AIKE,E211900B,496000.00,02USSHY26SPEC2201,...,100101.0,CHINA,NaN,INDUS,NET 45 DAYS,FOB NY PORT,92059e56-e620-40ac-e111-e6c2e3812162,1001-1,NaN,NaN
2,1002,1,4686,2003-12-19,Chg,1,AICE,E311603B-1,320666.67,SMJR2BOO11,...,31961.0,SAUDI ARABIA,NaN,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2e6812162,1002-1,NaN,NaN
3,1002,2,4691,2003-12-19,Chg,1,AICE,E311603B-2,320666.67,SMJR2BOO11,...,31961.0,SAUDI ARABIA,NaN,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2eb812162,1002-2,NaN,NaN
4,1002,3,4692,2003-12-19,New,1,AICE,E311603B-3,320666.66,SMJR2BOO11,...,31961.0,SAUDI ARABIA,NaN,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2ec812162,1002-3,NaN,NaN
5,1003,2,4668,2003-10-14,Chg,1,AIAS,E311763EF,34460.00,JV03-16AU,...,201612.0,CHINA,NaN,STKPKGPTS,NET 90 DAYS,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2f4812162,1003-2,a1ni0000000JCikAAG,NaN
6,1003,3,4671,2003-10-14,Chg,1,AIAS,E311763EF,51280.00,JV03-16AU,...,201612.0,CHINA,NaN,STKPKGPTS,NET 90 DAYS,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2f7812162,1003-3,a1ni0000000JCikAAG,NaN
7,1004,1,4672,2003-10-02,New,1,AISE,C611501,90000.00,10419,...,409835.0,SINGAPORE,NaN,NaN,NET 30 DAYS,FOB East Coast,92059e56-e620-40ac-e111-e6c2f8812162,1004-1,a1ni0000000JCeZAAW,02ii0000006PuhsAAC
8,1004,2,4673,2003-10-02,New,1,AISE,C611501,82765.00,10419,...,409835.0,SINGAPORE,NaN,NaN,NET 30 DAYS,FOB East Coast,92059e56-e620-40ac-e111-e6c2f9812162,1004-2,a1ni0000000JCeZAAW,02ii0000006PuhsAAC
9,1005,1,4674,2003-10-02,New,1,AICE,E311619,243500.00,PAS-EL-01/2003,...,10160.0,THAILAND,NaN,ENG,NET 30 DAYS,CIF BANGKOK,92059e56-e620-40ac-e111-e6c2fa812162,1005-1,a1ni0000000JDe9AAG,02ii0000006PvZaAAK


In [4]:
sfdc_df

,OrderNum__c,OrderLn__c,Name,Book_Date__c,BookType__c,OrderRel__c,Prod_Code__c,SN__c,Order_Line_Amt__c,PO_Num__c,...,Ship_To_Zip__c,Ship_To_Country__c,Packager__c,Market_Segment__c,Payment_Terms__c,OrderHed_FOB__c,SysRowID__c,Order_Num_Ln__c,Factory_Order__c,Asset__c
0,1000.0,1.0,4650,2003-12-30,Void,1.0,AIIE,E110042-3,0.00,A009-41003 R1,...,None,INDONESIA,None,None,XLETTER OF CRED,C&F JAKARTA,92059e56-e620-40ac-e111-e6c2e2812162,1000-1,None,02ii0000006PvLyAAK
1,1001.0,1.0,4651,2003-10-02,New,1.0,AIKE,E211900B,496000.00,02USSHY26SPEC2201,...,100101.0,CHINA,None,INDUS,NET 45 DAYS,FOB NY PORT,92059e56-e620-40ac-e111-e6c2e3812162,1001-1,None,None
2,1002.0,1.0,4654,2003-12-19,Chg,1.0,AICE,E311603B-1,320666.67,SMJR2BOO11,...,31961.0,SAUDI ARABIA,None,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2e6812162,1002-1,None,None
3,1002.0,2.0,4659,2003-12-19,Chg,1.0,AICE,E311603B-2,320666.67,SMJR2BOO11,...,31961.0,SAUDI ARABIA,None,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2eb812162,1002-2,None,None
4,1002.0,3.0,4662,2003-12-19,Chg,1.0,AICE,E311603B-3,320666.66,SMJR2BOO11,...,31961.0,SAUDI ARABIA,None,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2ee812162,1002-3,None,None


In [5]:
merged_df = bookings_df.append(sfdc_df)

In [6]:
merged_df

,OrderNum__c,OrderLn__c,Name,Book_Date__c,BookType__c,OrderRel__c,Prod_Code__c,SN__c,Order_Line_Amt__c,PO_Num__c,...,Ship_To_Zip__c,Ship_To_Country__c,Packager__c,Market_Segment__c,Payment_Terms__c,OrderHed_FOB__c,SysRowID__c,Order_Num_Ln__c,Factory_Order__c,Asset__c
0,1000.0,1.0,4682,2003-12-30 00:00:00,Void,1.0,AIIE,E110042-3,0.00,A009-41003 R1,...,NaN,INDONESIA,NaN,NaN,XLETTER OF CRED,C&F JAKARTA,92059e56-e620-40ac-e111-e6c2e2812162,1000-1,NaN,02ii0000006PvLyAAK
1,1001.0,1.0,4683,2003-10-02 00:00:00,New,1.0,AIKE,E211900B,496000.00,02USSHY26SPEC2201,...,100101,CHINA,NaN,INDUS,NET 45 DAYS,FOB NY PORT,92059e56-e620-40ac-e111-e6c2e3812162,1001-1,NaN,NaN
2,1002.0,1.0,4686,2003-12-19 00:00:00,Chg,1.0,AICE,E311603B-1,320666.67,SMJR2BOO11,...,31961,SAUDI ARABIA,NaN,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2e6812162,1002-1,NaN,NaN
3,1002.0,2.0,4691,2003-12-19 00:00:00,Chg,1.0,AICE,E311603B-2,320666.67,SMJR2BOO11,...,31961,SAUDI ARABIA,NaN,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2eb812162,1002-2,NaN,NaN
4,1002.0,3.0,4692,2003-12-19 00:00:00,New,1.0,AICE,E311603B-3,320666.66,SMJR2BOO11,...,31961,SAUDI ARABIA,NaN,ENG,PROGRESS PMT 45,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2ec812162,1002-3,NaN,NaN
5,1003.0,2.0,4668,2003-10-14 00:00:00,Chg,1.0,AIAS,E311763EF,34460.00,JV03-16AU,...,201612,CHINA,NaN,STKPKGPTS,NET 90 DAYS,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2f4812162,1003-2,a1ni0000000JCikAAG,NaN
6,1003.0,3.0,4671,2003-10-14 00:00:00,Chg,1.0,AIAS,E311763EF,51280.00,JV03-16AU,...,201612,CHINA,NaN,STKPKGPTS,NET 90 DAYS,FOB PORT NEW YK,92059e56-e620-40ac-e111-e6c2f7812162,1003-3,a1ni0000000JCikAAG,NaN
7,1004.0,1.0,4672,2003-10-02 00:00:00,New,1.0,AISE,C611501,90000.00,10419,...,409835,SINGAPORE,NaN,NaN,NET 30 DAYS,FOB East Coast,92059e56-e620-40ac-e111-e6c2f8812162,1004-1,a1ni0000000JCeZAAW,02ii0000006PuhsAAC
8,1004.0,2.0,4673,2003-10-02 00:00:00,New,1.0,AISE,C611501,82765.00,10419,...,409835,SINGAPORE,NaN,NaN,NET 30 DAYS,FOB East Coast,92059e56-e620-40ac-e111-e6c2f9812162,1004-2,a1ni0000000JCeZAAW,02ii0000006PuhsAAC
9,1005.0,1.0,4674,2003-10-02 00:00:00,New,1.0,AICE,E311619,243500.00,PAS-EL-01/2003,...,10160,THAILAND,NaN,ENG,NET 30 DAYS,CIF BANGKOK,92059e56-e620-40ac-e111-e6c2fa812162,1005-1,a1ni0000000JDe9AAG,02ii0000006PvZaAAK


In [9]:
merged_df = merged_df.astype({'Name': 'int32'})

In [10]:
merged_df.dtypes

OrderNum__c                         float64
OrderLn__c                          float64
Name                                  int32
Book_Date__c                         object
BookType__c                          object
OrderRel__c                         float64
Prod_Code__c                         object
SN__c                                object
Order_Line_Amt__c                   float64
PO_Num__c                            object
Need_By_Date__c                      object
Quote_Num__c                         object
Quote_Line__c                        object
Sales_Representative__c              object
Salesperson__c                       object
Primary_Salesperson_Origin__c        object
Sales_Director_Origin__c             object
Regional_Sales_Manager_Origin__c     object
Cust_ID__c                           object
Customer_Name__c                     object
Ultimate_User__c                     object
Bill_To_Name__c                      object
Bill_To_Address_1__c            